In [ ]:
import pandas as pd
import numpy as np
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

MODEL_NAME = ""
CSV_FILE_PATH = "C:\\Users\\SMHRD\\Documents\\카카오톡 받은 파일\\참조자료v3.csv"

class ResponseSearch():
    def __init__(self,csv_file_path,model_name):
        self.csv_file_path = csv_file_path
        self.data = self._load_data()
        self.tokenizer = AutoTokenizer.from_pretrained(csv_file_path)
        self.model = AutoModel.from_pretrained(model_name)

        self.preprocess_embedding()

    def _load_data(self):
        data = pd.read_csv(self.csv_file_path)
        return data

    def str_to_vector(embedding_str):
        return np.fromstring(embedding_str, sep=",")
    
    def preprocess_embedding(self):
        #대처_임베딩의 백터값을 numpy화 : 코사인유사도 비교를 위함.
        self.data["대처_임베딩"] = self.data["대처_임베딩_백터"].apply(str_to_vector)
        



    #텍스트 임베딩    
    def embed_text(self,text):
        inputs = self.tokenizer(text,return_tensor="pt",truncation=True,padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state[:,0,:].squeeze()
        return embeddings.numpy()

    #특정 고민유형의 임베딩 데이터 불러오기
    def get_cate_embed(self,reason):
        cate_data = self.data[self.data["유형설명"]== reason]
        #고민 유형별로 수많은 row를 numpy형식으로 한줄로 변경
        cate_embeddings = np.array([np.fromstring(emb,sep=',') for emb in cate_data["원인_임베딩값"]])
        return cate_embeddings,cate_data
    
    #가장 적합한 답변 찾는 방법
    def find_best_ans(self,user_input,reason):
        # 사용자 질문 embedding
        user_emd = self.embed_text(user_input)

        #특정 고민유형의 임베딩데이터 로드
        cate_embedding, cate_data = self.get_cate_embed(reason)

        # 유사도 계산
        similar = cosine_similarity(user_emd.reshape(1,-1),cate_embedding)
        
        # 가장 유사도 높은 임베딩의 인덱스찾기
        most_sim_idx = np.argmax(similar)

        # 가장 유사도 높은 답변데이터
        most_sim_entry = cate_data.iloc[most_sim_idx]["답변"]
        
        return most_sim_entry
